# Dirac-3 Continuous

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial03-continuous.ipynb)

## Encoding

To encode variables in continuous values, Dirac-3 uses photon count in a time bin as a linearly scaled representation of the value for the qudit associated with that time bin. To obtain the qudit values, the photon counts across all qudits are normalized to a total equal to the sum constraint value. Where $R$ is the sum constraint value, $c_i$ is the photon count for time bin $i$, 
$$
x_i = R \left(\frac{c_i}{\sum_j c_j}\right)
$$
The details on how the photon counts can represent this are proprietary. Each qudit is represented by a discrete number of levels. This number of levels subject to the sensitivity of the device, given environmental changes. The minimum number of levels is 200, which directly corresponds to the minimum sensitivity of the device in decibels, approximately 23 dB. The max sensitivity that Dirac-3 can achieve currently is 40 dB. This corresponds to 10,000 levels. 

## Sum Constraint
In the previous tutorial, the `sum_constraint` parameter was introduced.
$$
\sum_i x_i = R
$$

## Distillation
There isn't an error correction scheme needed for this computing paradigm like there is for gate model devices. The EQC does have to mitigate shot noise and the output values could have an error rate of up to $\sqrt{c_i}$. This provides an opportunity for post-processing that we call distillation, done using sampling from distributions based on the state vector ($(c_0, c_1,\ldots,c_{n-1})$) given by the device. Distillation is triggered by requesting a solution precision.

## Supported Polynomial
Dirac-3 has the capability of representing up to fifth-order terms:
$$
E(x)=\sum_i C_i x_i + \sum_i\sum_j J_{ij}x_ix_j + \sum_i\sum_j\sum_k T_{ijk} x_ix_jx_k + \sum_i\sum_j\sum_k\sum_l Q_{ijkl} x_ix_jx_kx_l+ \sum_i\sum_j\sum_k\sum_l\sum_m P_{ijklm} x_ix_jx_kx_lx_m .
$$

## Device Configuration Parameters

Dirac-3 supports three parameters which directly impact the evolution of a solution sample.

- `relaxation_schedule` is a parameter which represents the choice of one of 4 preconfigured parameter settings. 1 is a choice that also includes an artificial limit on the number of iterations. For the options 2, 3 and 4, the higher the number, the higher the probability of finding the ground state in a single sample. Trade-offs between the number of samples requested and relaxation schedule can lead to more efficient sampling.
- `mean_photon_number` sets the average number of photons in every quantum state. Allowed values are floating point numbers between 0.1 and 2.
- `quantum_fluctuation_coefficient` sets the amount of loss introduced into the system for each loop in the measurement process. Allowed values are integers between 1 and 50.


In [ ]:
# If using this in Google Colab, we need to install the required packages
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !pip install eqc-models==0.12.0 # The version may change, check the latest on PyPI

In [1]:
import numpy as np
from eqc_models.base import PolynomialModel
from eqc_models.solvers import Dirac3ContinuousCloudSolver

## Cubic Example
$$
E(x) = -0.75x_1^2-0.25x_2^2-x_3^2 + x_1x_2x_3
$$

In [2]:
R = 2
coefficients = [-0.75, -0.25, -1, 1]
indices = [(0, 1, 1), (0, 2, 2), (0, 3, 3), (1, 2, 3)]
model = PolynomialModel(coefficients, indices)
model.upper_bound = R*np.ones((3,))

In [3]:
solver = Dirac3ContinuousCloudSolver()
response = solver.solve(model, sum_constraint=2, relaxation_schedule=1, num_samples=5)
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-03-14 22:19:33 - Dirac allocation balance = 0 s (unmetered)
2025-03-14 22:19:33 - Job submitted: job_id='67d4ffd500e804f113aefc98'
2025-03-14 22:19:33 - QUEUED
2025-03-14 22:19:36 - RUNNING
2025-03-14 22:20:17 - COMPLETED
2025-03-14 22:20:20 - Dirac allocation balance = 0 s (unmetered)
[[0, 0, 2], [1e-07, 0, 2], [2, 0, 0], [2, 1e-07, 0]]
[-4, -4, -3, -3]
[2, 1, 1, 1]


In [4]:
solver = Dirac3ContinuousCloudSolver()
response = solver.solve(model, sum_constraint=2, relaxation_schedule=1, num_samples=10, mean_photon_number=0.8)
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-03-14 22:20:23 - Dirac allocation balance = 0 s (unmetered)
2025-03-14 22:20:24 - Job submitted: job_id='67d5000800e804f113aefc9a'
2025-03-14 22:20:24 - QUEUED
2025-03-14 22:21:03 - RUNNING
2025-03-14 22:21:13 - COMPLETED
2025-03-14 22:21:16 - Dirac allocation balance = 0 s (unmetered)
[[1e-07, 0, 2], [2, 1e-07, 0], [2, 0, 0], [2, 1e-07, 1e-07], [2, 0, 1e-07], [0, 2, 0]]
[-4, -3, -3, -3, -3, -1]
[1, 3, 2, 2, 1, 1]


In [5]:
solver = Dirac3ContinuousCloudSolver()
response = solver.solve(model, sum_constraint=2, relaxation_schedule=1, num_samples=5, mean_photon_number=0.1)
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-03-14 22:21:18 - Dirac allocation balance = 0 s (unmetered)
2025-03-14 22:21:18 - Job submitted: job_id='67d5003e00e804f113aefc9c'
2025-03-14 22:21:18 - QUEUED
2025-03-14 22:22:24 - RUNNING
2025-03-14 22:22:34 - COMPLETED
2025-03-14 22:22:37 - Dirac allocation balance = 0 s (unmetered)
[[1e-07, 0, 2], [2, 1e-07, 0]]
[-4, -3]
[1, 4]


In [6]:
for sol in response["results"]["solutions"]:
    print(model.polynomial.evaluate(np.array(sol)))

[-4.000000000000007]
[-3.0000000000000027]


## Slack Qudit
For most problems, the sum of all variables is not known beforehand. To support this requirement, a "slack" qudit can be used.

This polynomial is minimized with $x_1=0,x_2=0$.

$$
E(x)=x_1+x_2+x_1x_2
$$

In [7]:
coefficients = [1, 1, 1]
indices = [(0, 1), (0, 2), (1, 2)]
model = PolynomialModel(coefficients, indices)
model.upper_bound = np.ones((2,))
model.machine_slacks = 1

Notice the extra value in the solution.

In [8]:
response = solver.solve(model, sum_constraint=1, num_samples=5, relaxation_schedule=1)
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-03-14 22:22:39 - Dirac allocation balance = 0 s (unmetered)
2025-03-14 22:22:39 - Job submitted: job_id='67d5008f00e804f113aefc9e'
2025-03-14 22:22:40 - QUEUED
2025-03-14 22:22:55 - RUNNING
2025-03-14 22:23:34 - COMPLETED
2025-03-14 22:23:37 - Dirac allocation balance = 0 s (unmetered)
[[0, 0, 1]]
[0]
[5]
